In [ ]:
%matplotlib nbagg
# Libraries
import scipy as sc
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import matplotlib
from   IPython.display import Markdown, display
#
from   repropy import Context, Meta, Tagged
import analysis
import analysis.plots as plot
from analysis.numass.meta import *
#
import unfolding as lib

In [ ]:
meta = Meta(
    dataset = dataset_2011 + Meta(
        drop_last=8,
    ),
    basis = Meta(
        oversample = 2,
    ),
    transmission = 'linear',
    omega        = [Tagged('OmegaIntSpec', deg=2, equalize=True)],
)
ctx = Context(meta, store_dir='../store')
aa  = analysis.analytic.Analytic(ctx)
aPosterior = analysis.analytic.AlphaPosterior(aa.unfold.value(), aa.alpha.value())

# Simple model without constraints

In [ ]:
model = pm.Model()
with model :
    # --------------------------------------
    # Regularization strength
    # --------------------------------------
    # alpha = pm.HalfFlat('alpha')   
    alphaLN = aPosterior.lognormal(scale=3)
    alpha   = pm.Lognormal('alpha', mu=alphaLN.mu, sd=alphaLN.sig)
    # --------------------------------------
    # φ
    # --------------------------------------
    nPhi  = len(aa.basis.value())
    om    = aa.unfold.value().omegas[0].mat
    chol  = np.linalg.cholesky(np.linalg.inv(om))
    phi   = pm.MvNormal('phi',
                        mu    = np.zeros(nPhi),
                        chol  = chol/np.sqrt(alpha),
                        shape = nPhi
                       )
    # --------------------------------------
    # Experimental data
    # --------------------------------------
    f     = pm.Normal('f', 
                      mu       = pm.math.dot(aa.unfold.value().K, phi),
                      sd       = aa.dat.value()['err'].values, 
                      shape    = len(aa.dat.value()),
                      observed = aa.dat.value()['cnt'].values,
                     )

In [ ]:
with model :
    trace = pm.sample(draws=1000, cores=8)

In [ ]:
pm.summary(trace).round(3)

In [ ]:
pm.traceplot(trace)
None

In [ ]:
fig = plot.plot_alpha(aa.unfold.value(), aPosterior)
dens,bins = np.histogram(trace['alpha'], bins=50, density=True)
dens = np.log(dens)
dens = dens - np.max(dens)
fig.axes[0].set_ylim((-2, 0.1))
plt.bar(x=bins[1:], height=dens+2, width=np.diff(bins), bottom=-2, align="edge", fc=(0.1,0.1,0.8,0.3))
plt.semilogx()
None

In [ ]:
plot.plot_measurements_mcmc(aa, trace)

In [ ]:
plot.plot_deconvolved_mcmc(aa, trace)
None

In [ ]:
plot.plot_correlations_matrix( np.cov(trace['phi'], rowvar=False))
None